In [ ]:
from pyxtension.streams import stream
import re

file = "aoc-7.ex1.txt"
file = "aoc-7.ex2.txt"

In [ ]:
with open(file) as file:
    lines = [line.rstrip() for line in file]


In [ ]:
def create_directory():
    return ({}, [])

def is_cd(line):
    return line.startswith('$ cd ')
def is_ls(line):
    return line.startswith('$ ls')

def is_dir_ls(line):
    return line.startswith('dir ')

def create_file(line):
    parts = line.split()
    name = parts[1]
    size = int(parts[0])
    return (name, size)

def create_directory_if_not_exists(path):
    global filesystem
    current_directory = filesystem
    for element in path[1:]:
        if element not in current_directory[0]:
            dir = create_directory()
            current_directory[0][element] = dir
        current_directory = current_directory[0][element]

def change_directory(line):
    global current_path
    navigation = line.split('$ cd ')[1]
    if (navigation == '/'):
        current_path = ['/']
    elif navigation == '..':
        current_path = current_path[:-1]
    else:
        current_path.append(navigation)
    create_directory_if_not_exists(current_path)

def add_directory(line):
    global filesystem
    global current_path
    name = line.split()[1]
    current_directory = filesystem
    for element in current_path[1:]:
        current_directory = current_directory[0][element]
    dir = create_directory()
    current_directory[0][name] = dir

def add_file(line):
    global filesystem
    global current_path
    file = create_file(line)
    current_directory = filesystem
    for element in current_path[1:]:
        current_directory = current_directory[0][element]
    current_directory[1].append(file)

filesystem = create_directory()
current_path = ['/']

for line in lines:
    if is_cd(line):
        change_directory(line)
    elif is_ls(line):
        pass
    elif is_dir_ls(line):
        add_directory(line)
    else:
        add_file(line)

### Part 1

In [ ]:
def get_sizes(name, directory):
    dir_sizes = stream(directory[0])\
        .flatMap(lambda n: get_sizes(n, directory[0][n]))\
        .toList()
    file_size = stream(directory[1])\
        .map(lambda f: f[1])\
        .sum()
    contained_dir_size = stream(dir_sizes)\
        .filter(lambda s: s[2])\
        .map(lambda s: s[1])\
        .sum()
    cleared_dir_sizes =  stream(dir_sizes)\
        .map(lambda s: (s[0], s[1], False))\
        .toList()
    return [(name, file_size + contained_dir_size, True)] + cleared_dir_sizes

sizes = get_sizes('/', filesystem)

stream(sizes)\
    .map(lambda s: s[1])\
    .filter(lambda s: s <= 100000)\
    .sum()

### Part 2

In [ ]:
disk_size = 70000000
needed_size = 30000000

used_size = stream(sizes)\
    .map(lambda s: s[1])\
    .sorted(reverse=True)\
    .head(1)\
    .sum()

free_size = used_size - (disk_size - needed_size)

stream(sizes)\
    .map(lambda s: s[1])\
    .sorted()\
    .filter(lambda s: s > free_size)\
    .head(1)\
    .sum()